# Build Package Command

In [1]:
%load_ext literary.notebook

In [2]:
import pathlib

from traitlets import List, Unicode, default

from ..core.package import build_package
from .app import LiteraryApp

In [3]:
class LiteraryBuildApp(LiteraryApp):
    description = "Build a pure-Python package from a set of Jupyter notebooks"
    aliases = {
        **LiteraryApp.aliases,
        "source": "LiteraryBuildApp.source",
        "package": "LiteraryBuildApp.package",
        "ignore": "LiteraryBuildApp.ignore",
    }

    source = Unicode(help="source directory for notebooks").tag(config=True)
    package = Unicode(help="destination path generated package").tag(config=True)
    ignore = List(help="glob pattern to ignore during recursion", trait=Unicode()).tag(
        config=True
    )

    def start(self):
        if not self.source:
            raise ValueError(f"Missing source path")
        source = pathlib.Path(self.source)

        if not self.package:
            raise ValueError(f"Missing package path")
        package = pathlib.Path(self.package)

        build_package(source, package, self.ignore)